# scrapy中Item Pipelines用法

当Item 在Spider中被收集之后，就会被传递到Item Pipeline中进行处理

每个item pipeline组件是实现了简单的方法的python类，负责接收到item并通过它执行一些行为，

同时也决定此Item是否继续通过pipeline,或者被丢弃而不再进行处理

item pipeline的主要作用：
1. 清理html数据
2. 验证爬取的数据
3. 去重并丢弃
4. 将爬取的结果保存到数据库中或文件中

## 编写自己的pipeline

* process_item(self,item,spider):
    * 每个item piple组件是一个独立的pyhton类，必须实现`process_item(self,item,spider)`方法
    * 每个item pipeline组件都需要调用该方法，这个方法必须返回一个具有数据的dict,或者item对象，或者抛出DropItem异常，被丢弃的item将不会被之后的pipeline组件所处理

下面的方法也可以选择实现

* open_spider(self,spider):表示当spider被开启的时候调用这个方法

* close_spider(self,spider):当spider关闭时这个方法被调用

* from_setings(cls,settings):这个和前面说spider的时候用法是一样，用于获取settings配置文件中的信息，注意这是一个类方法.

## 一些pipeline使用例子

这个例子是判断item中是否包含price以及price_excludes_vat，如果存在则调整price属性，让`item['price'] = item['price'] * self.vat_factor`，如果不存在则返回DropItem

In [ ]:
from scrapy.exceptions import DropItem

class PricePipeline(object):

    vat_factor = 1.15

    def process_item(self, item, spider):
        if item['price']:
            if item['price_excludes_vat']:
                item['price'] = item['price'] * self.vat_factor
            return item
        else:
            raise DropItem("Missing price in %s" % item)

### 将item写入到json文件中

In [ ]:
import json

class JsonWriterPipeline(object):

    def __init__(self):
        self.file = open('items.jl', 'wb')

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

### 将item写入到MongoDB

同时这里演示了from_crawler的用法

In [ ]:
import pymongo

class MongoPipeline(object):

    collection_name = 'scrapy_items'

    def __init__(self, mongo_uri, mongo_db):
        self.mongo_uri = mongo_uri
        self.mongo_db = mongo_db

    @classmethod
    def from_crawler(cls, crawler):
        return cls(
            mongo_uri=crawler.settings.get('MONGO_URI'),
            mongo_db=crawler.settings.get('MONGO_DATABASE', 'items')
        )

    def open_spider(self, spider):
        self.client = pymongo.MongoClient(self.mongo_uri)
        self.db = self.client[self.mongo_db]

    def close_spider(self, spider):
        self.client.close()

    def process_item(self, item, spider):
        self.db[self.collection_name].insert(dict(item))
        return item

### 去重

一个用于去重的过滤器，丢弃那些已经被处理过的item,假设item有一个唯一的id，但是我们spider返回的多个item中包含了相同的id,去重方法如下：这里初始化了一个集合，每次判断id是否在集合中已经存在，从而做到去重的功能

In [ ]:
from scrapy.exceptions import DropItem

class DuplicatesPipeline(object):

    def __init__(self):
        self.ids_seen = set()

    def process_item(self, item, spider):
        if item['id'] in self.ids_seen:
            raise DropItem("Duplicate item found: %s" % item)
        else:
            self.ids_seen.add(item['id'])
            return item

### 启用一个item Pipeline组件

在settings配置文件中有一个ITEM_PIPELINES的配置参数，例子如下：

```python
ITEM_PIPELINES = {
    'myproject.pipelines.PricePipeline': 300,
    'myproject.pipelines.JsonWriterPipeline': 800,
}
```

每个pipeline后面有一个数值，这个数组的范围是0-1000，这个数值确定了他们的运行顺序，数字越小越优先